In [22]:
import os
import pandas as pd
import unidecode
from datetime import time
import numpy as np


In [9]:
# Définition du dossier de données et du préfixe des fichiers
csv_folder = "raw_data"  # à adapter si besoin
file_prefix = "EXP_"  # préfixe des fichiers CSV

In [10]:
def load_csv_files(csv_folder, file_prefix="EXP_", sep=';'):
    """
    Charge tous les fichiers CSV dans un dossier donné, les convertit en DataFrames, et les stocke dans un dictionnaire.

    Parameters:
        csv_folder (str): Chemin du dossier contenant les fichiers CSV.
        file_prefix (str): Préfixe des fichiers CSV à charger (par défaut : "EXP_").
        sep (str): Séparateur utilisé dans les fichiers CSV (par défaut : ';').

    Returns:
        dict: Dictionnaire contenant les DataFrames, avec les noms de fichiers comme clés.
    """
    dataframes = {}
    # Liste tous les fichiers dans le dossier qui commencent par le préfixe et se terminent par .csv
    files = [f for f in os.listdir(csv_folder) if f.startswith(file_prefix) and f.endswith(".csv")]

    # Charger chaque fichier dans un DataFrame
    for file in files:
        # Utiliser le nom de fichier (sans préfixe ni extension) comme clé du dictionnaire
        df_name = file.replace(file_prefix, "").replace(".csv", "").capitalize()
        df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)

        dataframes[df_name] = df

    return dataframes


# Chemin du dossier contenant les fichiers CSV
csv_folder_path = "../../../raw_data/Daudet/2023-2024/Pronote/"
# Charger les fichiers
data_pronote = load_csv_files(csv_folder=csv_folder_path)


/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_72621/1371040477.py:21: DtypeWarning: Columns (91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,121,122,123,124,126,127,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)


In [11]:
# Dictionnaire des colonnes à conserver et à renommer pour chaque DataFrame
columns_config = {
    'Punition': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PUNITION', 'DATE', 'MOTIF', 'HEURE']
    },
    'Notesdevoir': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'MOYENNE', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}
    },
    'Professeur': {
        'keep': ['IDENT', 'CIVILITE', 'NOM', 'PRENOM']
    },
    'Absenceseleves': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'DATE DEBUT', 'DEMI JOUR'],
        'rename': {'DATE DEBUT': 'DATE', 'DEMI JOUR': 'NB_DEMI_JOURNEE'}
    },
    'Absencerepas': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE']
    },
    'Retards': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'REGLE', 'DATE', 'HEURE', 'DUREE']
    },
    'Eleve': {
        'keep': ['IDENT', 'NOM', 'PRENOM', 'DATE NAISS', 'CLASSES', 'SEXE'],
        'rename': {'DATE NAISS': 'DATE_NAISS'}
    },
    'Service': {
        'keep': ['MATIERE', 'CLASSES', 'PROFS', 'NB DEVOIRS', 'COEFF'],
        'rename': {'NB DEVOIRS': 'NB_DEVOIRS'}
    },
    'Appreciationprofesseurs': {
        'keep': ['NOM', 'PRENOM', 'MATIERE', 'CLASSE/GRPE', 'PROFS', 'PERIODE', 'APPR.A'],
        'rename': {'CLASSE/GRPE': 'CLASSE', 'APPR.A': 'APPR'}
    },
    'Passagesinfirmerie': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'H DEBUT', 'DATE', 'DUREE'],
        'rename': {'H DEBUT': 'HEURE'}
    },
    'Appreciationduconseil': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PERIODE', 'APPR.C'],
        'rename': {'APPR.C': 'APPR'}
    },
    'Sanction': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE', 'SANCTION', 'MOTIF']
    },
    'Notes': {
        'keep': ['PROFS', 'MATIERE', 'CLASSE/GRPE', 'NOM ELEVE', 'PRENOM ELEVE', 'MOY_ELEVE'] +
                [f'Devoir{i} - Note' for i in range(1, 25)] +
                [f'Devoir{i} - Coeff' for i in range(1, 25)] +
                [f'Devoir{i} - Date' for i in range(1, 25)] +
                [f'Devoir{i} - Période Notation' for i in range(1, 25)],
                'rename': {'NOM ELEVE': 'NOM', 'PRENOM ELEVE':'PRENOM'},
        'drop_pattern': 'Devoir\d+ - Facultatif'
    }
}

# Fonction de sélection et de renommage des colonnes
def select_and_rename_columns(dataframes, config):
    for name, settings in config.items():
        if name in dataframes:
            df = dataframes[name]

            # Sélectionner les colonnes à conserver
            if 'keep' in settings:
                df = df[settings['keep']]

            # Renommer les colonnes
            if 'rename' in settings:
                df.rename(columns=settings['rename'], inplace=True)

            # Supprimer les colonnes correspondant à un motif si spécifié
            if 'drop_pattern' in settings:
                drop_cols = df.filter(regex=settings['drop_pattern']).columns
                df.drop(columns=drop_cols, inplace=True)

            dataframes[name] = df

# Appliquer la sélection et le renommage des colonnes
select_and_rename_columns(data_pronote, columns_config)


/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_72621/2073051042.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=settings['rename'], inplace=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_72621/2073051042.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=settings['rename'], inplace=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_72621/2073051042.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [12]:
# Fonction de nettoyage et de formatage des noms de colonnes
def clean_column_names(dataframes):
    for name, df in dataframes.items():
        # Normaliser les noms de colonnes
        df.columns = [unidecode.unidecode(col).lower().replace(" ", "_").replace("-", "_") for col in df.columns]
        dataframes[name] = df

# Appliquer le nettoyage des noms de colonnes
clean_column_names(data_pronote)

In [13]:
# Dictionnaire pour catégoriser les colonnes par types de données attendus
column_types = {
    'string': ['nom', 'prenom', 'classes', 'matiere', 'profs', 'civilite', 'sexe', 'motif', 'punition', 'regle'],
    'float': ['note', 'coeff', 'sur', 'moyenne'],
    'int': ['nb_demi_journee', 'nb_devoirs'],
    'date': ['date', 'date_naiss', 'periode', 'devoir1___date', 'devoir2___date', 'devoir3___date'],  # etc.
    'time': ['heure', 'duree'],  # Les durées ou heures à normaliser en type `time`
    'category': ['classe', 'trimestre']
}


# Fonction d'ajustement des conversions pour les colonnes restantes
def adjust_column_types(dataframes):
    for name, df in dataframes.items():

        # Conversion des colonnes de type `heure` et `duree` en format `time`
        time_cols = ['heure', 'duree']
        for col in time_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col].astype(str).str.replace('h', ':'), format='%H:%M', errors='coerce').dt.time

        # Uniformisation de la colonne `periode_notation` dans le DataFrame `Notes`
        if name == 'Notes':
            period_cols = [col for col in df.columns if 'periode_notation' in col]
            for col in period_cols:
                # Assumer une conversion en `category` si les valeurs sont discrètes ou en `float` si elles sont numériques
                if df[col].dtype == 'object':
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                df[col] = df[col].astype('category') if df[col].nunique() < 10 else df[col]

        # Mettre à jour le DataFrame dans le dictionnaire
        dataframes[name] = df

# Appliquer les ajustements
adjust_column_types(data_pronote)


# Test : Vérifier les types de données après conversion
for name, df in data_pronote.items():
    print(f"Types de données pour le DataFrame '{name}' après conversion finale :\n{df.dtypes}\n")


Types de données pour le DataFrame 'Punition' après conversion finale :
nom         object
prenom      object
classes     object
punition    object
date        object
motif       object
heure       object
dtype: object

Types de données pour le DataFrame 'Notesdevoir' après conversion finale :
classes      object
nom          object
prenom       object
profs        object
matiere      object
date         object
trimestre    object
sur          object
moyenne      object
coeff        object
note         object
dtype: object

Types de données pour le DataFrame 'Professeur' après conversion finale :
ident       object
civilite    object
nom         object
prenom      object
dtype: object

Types de données pour le DataFrame 'Absenceseleves' après conversion finale :
nom                object
prenom             object
classes            object
motif              object
date               object
nb_demi_journee    object
dtype: object

Types de données pour le DataFrame 'Absencerepas' après 

In [20]:
import os
import pandas as pd
import unidecode

# Charger les fichiers CSV
def load_csv_files(csv_folder, file_prefix="EXP_", sep=';'):
    dataframes = {}
    files = [f for f in os.listdir(csv_folder) if f.startswith(file_prefix) and f.endswith(".csv")]
    for file in files:
        df_name = file.replace(file_prefix, "").replace(".csv", "").capitalize()
        df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)
        dataframes[df_name] = df
    return dataframes

# Sélection et renommage des colonnes
def select_and_rename_columns(dataframes, config):
    for name, settings in config.items():
        if name in dataframes:
            df = dataframes[name]
            if 'keep' in settings:
                df = df[settings['keep']]
            if 'rename' in settings:
                df.rename(columns=settings['rename'], inplace=True)
            if 'drop_pattern' in settings:
                drop_cols = df.filter(regex=settings['drop_pattern']).columns
                df.drop(columns=drop_cols, inplace=True)
            dataframes[name] = df

# Nettoyage des noms de colonnes
def clean_column_names(dataframes):
    for name, df in dataframes.items():
        df.columns = [unidecode.unidecode(col).lower().replace(" ", "_").replace("-", "_") for col in df.columns]
        dataframes[name] = df

# Conversion des types avec gestion spécifique pour la colonne `sur`
def convert_column_types(dataframes, column_types):
    for name, df in dataframes.items():
        for col in df.columns:
            # Conversion des colonnes string
            if col in column_types.get('string', []):
                df[col] = df[col].astype(str)

            # Conversion spécifique pour la colonne `sur`
            elif col == 'sur':
                # Extraire la valeur après '/' et convertir en entier
                df[col] = pd.to_numeric(df[col].astype(str).str.replace('/', ''), errors='coerce').astype('Int64')

            # Conversion des colonnes float
            elif col in column_types.get('float', []):
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

            # Conversion des colonnes int
            elif col in column_types.get('int', []) and col != 'sur':  # Exclure `sur` pour éviter un double traitement
                # Convertir d'abord en float pour traiter les valeurs décimales et les `NaN`
                temp_col = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

                # Vérifier si toutes les valeurs non `NaN` sont des entiers
                if (temp_col.dropna() % 1 == 0).all():
                    # Convertir en `Int64` en utilisant `.fillna(np.nan).astype('Int64')`
                    df[col] = temp_col.fillna(np.nan).astype('Int64')
                else:
                    # Si des valeurs décimales existent, laisser en `float`
                    df[col] = temp_col

            # Conversion des colonnes date
            elif col in column_types.get('date', []):
                df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

            # Conversion des colonnes time
            elif col in column_types.get('time', []):
                df[col] = pd.to_datetime(df[col].astype(str).str.replace('h', ':'), format='%H:%M', errors='coerce').dt.time

            # Conversion des colonnes category
            elif col in column_types.get('category', []):
                df[col] = df[col].astype('category')

        # Mettre à jour le DataFrame dans le dictionnaire
        dataframes[name] = df


In [ ]:
# Configurer la sélection et le renommage des colonnes
columns_config = {
    'Punition': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PUNITION', 'DATE', 'MOTIF', 'HEURE']
    },
    'Notesdevoir': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'MOYENNE', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}
    },
    'Professeur': {
        'keep': ['IDENT', 'CIVILITE', 'NOM', 'PRENOM']
    },
    'Absenceseleves': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'DATE DEBUT', 'DEMI JOUR'],
        'rename': {'DATE DEBUT': 'DATE', 'DEMI JOUR': 'NB_DEMI_JOURNEE'}
    },
    'Absencerepas': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE']
    },
    'Retards': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'REGLE', 'DATE', 'HEURE', 'DUREE']
    },
    'Eleve': {
        'keep': ['IDENT', 'NOM', 'PRENOM', 'DATE NAISS', 'CLASSES', 'SEXE'],
        'rename': {'DATE NAISS': 'DATE_NAISS'}
    },
    'Service': {
        'keep': ['MATIERE', 'CLASSES', 'PROFS', 'NB DEVOIRS', 'COEFF'],
        'rename': {'NB DEVOIRS': 'NB_DEVOIRS'}
    },
    'Appreciationprofesseurs': {
        'keep': ['NOM', 'PRENOM', 'MATIERE', 'CLASSE/GRPE', 'PROFS', 'PERIODE', 'APPR.A'],
        'rename': {'CLASSE/GRPE': 'CLASSE', 'APPR.A': 'APPR'}
    },
    'Passagesinfirmerie': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'H DEBUT', 'DATE', 'DUREE'],
        'rename': {'H DEBUT': 'HEURE'}
    },
    'Appreciationduconseil': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PERIODE', 'APPR.C'],
        'rename': {'APPR.C': 'APPR'}
    },
    'Sanction': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE', 'SANCTION', 'MOTIF']
    },
    'Notes': {
        'keep': ['PROFS', 'MATIERE', 'CLASSE/GRPE', 'NOM ELEVE', 'PRENOM ELEVE', 'MOY_ELEVE'] +
                [f'Devoir{i} - Note' for i in range(1, 25)] +
                [f'Devoir{i} - Coeff' for i in range(1, 25)] +
                [f'Devoir{i} - Date' for i in range(1, 25)] +
                [f'Devoir{i} - Période Notation' for i in range(1, 25)],
                'rename': {'NOM ELEVE': 'NOM', 'PRENOM ELEVE':'PRENOM'},
        'drop_pattern': 'Devoir\d+ - Facultatif'
    }
}

# Configurer les types de colonnes
column_types = {
    'string': ['nom', 'prenom', 'classes','profs', 'civilite', 'motif', 'punition', 'regle'],
    'float': ['note', 'moyenne'],
    'int': ['nb_demi_journee', 'nb_devoirs','coeff', 'sur'],
    'date': ['date', 'date_naiss', 'periode'],
    'time': ['heure', 'duree'],
    'category': ['classe', 'trimestre', 'matiere', 'sexe']
}


# Application
csv_folder_path = "../../../raw_data/Daudet/2023-2024/Pronote/"

#extraction des données
data_pronote = load_csv_files(csv_folder=csv_folder_path)

#selection des colonnes
select_and_rename_columns(data_pronote, columns_config)

#cleaning des noms de colonnes
clean_column_names(data_pronote)

#cleaning des types de colonnes
convert_column_types(data_pronote, column_types)

# Vérifier les types de données
for name, df in data_pronote.items():
    print(f"Types de données pour le DataFrame '{name}' après conversion finale :\n{df.dtypes}\n")